In [4]:
import pandas as pd
import numpy as np
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error,r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler
import argparse
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
from sklearn.pipeline import  Pipeline, make_pipeline
from sklearn.linear_model import 
from transformation import RemoveSkewnessKurtosis, Standardize

# Import Data
path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
df1 = pd.read_csv(path).iloc[:,1:]
df1.drop(columns = ['Veteran flag','Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)
X_admission1 = df1.drop(columns = ['Readmission', 'Died'])
Y_admission1 = df1[['Readmission']]

# Split Train and Test
X_train_ad1, X_test_ad1, y_train_ad1, y_test_ad1 = train_test_split(X_admission1, Y_admission1, test_size=0.20, random_state=42)

# Transform Data
targets = ['Readmission', 'Died']

cat_cols = ['AO', 'CVD', 'Ruca category encoded', 'Ethnicity', 
            'Gender', 'Races', 'Ethnicity_0',
            'Ethnicity_1', 'Ethnicity_2', 'Races_0', 
            'Races_1', 'Races_2', 'Races_3','DOMICILIARY', 
            'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY']

numeric_cols = ['num_stays', 'stay_length', 'num_unique_units',
       'num_transfers', 'num_cvd_readmission', 'unique_admitting_specialty', 
       'unique_discharging_specialty','Age 20-40', 'Age 40-60', 'Age 60-80', 'Age 80-100',
       'Age 100-120', 'age_mean', 'age_std', 'age_min', 'age_max', 'stay_min',
       'stay_max', 'stay_mean', 'stay_std', 'freq', 'total_procedure',
       'num_surgery_pro', 'num_immunization', 'Num med per admission mean',
       'Num med per admission min', 'Num med per admission max',
       'Total medications', 'mean age at specailty', 'period mean', 
       'specialty medical count', 'specialty support count',
       'period std','specialty count', 'Age 20-40 hypotension',
       'Age 40-60 hypotension', 'Age 60-80 hypotension',
       'Age 80-100 hypotension', 'Age 100-120 hypotension',
       'Age 20-40 hypertension', 'Age 40-60 hypertension',
       'Age 60-80 hypertension', 'Age 80-100 hypertension',
       'Age 100-120 hypertension', 'Age 20-40 healthy', 'Age 40-60 healthy',
       'Age 60-80 healthy', 'Age 80-100 healthy', 'Age 100-120 healthy',
       'lab_count', 'lab_freq', 'lab_age_mean', 'lab_age_std']

log_numeric_cols = ['num_stays_log', 'stay_length_log',
       'num_transfers_log', 'num_cvd_readmission_log',
       'unique_admitting_specialty_log', 'Age 20-40_log', 'Age 40-60_log',
       'Age 60-80_log', 'Age 80-100_log', 'Age 100-120_log', 'stay_min_log',
       'stay_max_log', 'stay_mean_log', 'stay_std_log', 'freq_log',
       'total_procedure_log', 'num_surgery_pro_log',
       'Num med per admission mean_log', 'Num med per admission min_log',
       'Num med per admission max_log', 'Total medications_log',
       'period mean_log', 'specialty medical count_log',
       'specialty support count_log', 'period std_log', 'specialty count_log',
       'Age 20-40 hypotension_log', 'Age 40-60 hypotension_log',
       'Age 60-80 hypotension_log', 'Age 80-100 hypotension_log',
       'Age 100-120 hypotension_log', 'Age 20-40 hypertension_log',
       'Age 40-60 hypertension_log', 'Age 60-80 hypertension_log',
       'Age 80-100 hypertension_log', 'Age 100-120 hypertension_log',
       'Age 20-40 healthy_log', 'Age 40-60 healthy_log',
       'Age 60-80 healthy_log', 'Age 80-100 healthy_log',
       'Age 100-120 healthy_log', 'lab_count_log', 'lab_freq_log']

transform_steps = [('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis(targets, cat_cols, numeric_cols, log_numeric_cols)),
         ('StandardizeStandardScaler', Standardize(log_numeric_cols, RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

data_prepared = transform_pipeline.transform(X_train_ad1)



/home/vivi/FDA/src/models/transformation.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X = X.rename(columns = {i:i+ "_rob_scaled"})


In [5]:
data_prepared

,Internalpatientid,num_stays_log_rob_scaled,stay_length_log_rob_scaled,num_transfers_log_rob_scaled,num_cvd_readmission_log_rob_scaled,unique_admitting_specialty_log_rob_scaled,Age 20-40_log_rob_scaled,Age 40-60_log_rob_scaled,Age 60-80_log_rob_scaled,Age 80-100_log_rob_scaled,...,Races_0,Races_1,Races_2,Races_3,DOMICILIARY,MEDICINE,NHCU,NON-COUNT,OTHERS,PSYCHIATRY
58864,117504,-0.203114,0.662921,0.693147,0.000000,0.000000,0.0,0.0,0.160558,0.000000,...,1,0,0,0,0,1,0,1,0,1
79350,158601,-0.834044,-1.191133,0.000000,0.000000,-0.756471,0.0,0.0,-0.226294,0.000000,...,1,0,0,0,0,0,0,0,0,0
16980,33850,-0.834044,0.080737,0.000000,0.000000,-0.756471,0.0,0.0,-0.613147,1.000000,...,0,1,0,0,0,1,0,0,0,0
66650,133029,-0.203114,-0.481701,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000,...,0,0,1,0,0,2,0,0,0,0
59734,119286,-0.834044,-0.027878,0.693147,0.000000,-0.756471,0.0,0.0,-0.226294,0.000000,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,12293,-0.464974,0.013631,0.693147,0.000000,-0.313964,0.0,0.0,-0.613147,1.584963,...,0,1,0,0,0,0,1,0,0,0
54886,109616,1.113928,0.838865,1.791759,2.321928,1.367211,0.0,0.0,0.773706,2.584963,...,0,1,0,0,0,13,0,1,2,0
76820,153543,0.869744,1.182991,0.693147,0.000000,0.610740,0.0,0.0,0.818378,0.000000,...,1,0,0,0,0,9,1,0,0,0
860,1629,-0.203114,0.440412,0.693147,1.000000,0.000000,0.0,0.0,0.160558,0.000000,...,0,1,0,0,0,2,1,0,0,0
